<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/learn_tf_009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning Part 2: Fine-Tuning

### What we will cover
1. Introduce fine-tuning, a transfer learning approach to modify pre-trained layers to be more suited for our data.
2. Using the keras Functional API
3. Smaller dataset of faster experimentation
4. Data Augmenetation ( how to make our training dataset more diverse without adding more data).
5. Running a series of model experiments using our Food Vision data
    1. Model0: A transfer learning model using Keras functional api
    2. Model1: A feature extraction transfer learning model using 1% of the data with data augmentation
    3. Model2: A feature extraction transfer learning model using 10% of the data with data augmentation.
    4. Model3: A fine-tuned transfer learning model on 10% of data.
    5. Model4: A fine-tuned transfer learning model on 100% of data.
6. Introduce model checkpoints to save intermediate model results
7. Compare the model experiments using tensorboard.




In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import datetime
print(f" Notebook last run (end-to-end) {datetime.datetime.now()}")

 Notebook last run (end-to-end) 2023-08-06 02:12:23.296755


In [3]:
import tensorflow as tf
print(f" Tensorflow version : {tf.__version__}")

 Tensorflow version : 2.12.0


In [4]:
%%writefile helper_functions.py
import matplotlib.pyplot as plt

def plot_loss_curve(history):
  """
    Plots the loss and accuracy curve for training and validation metrics
    Args:
      history(A History object):

  """
  epochs= range(len(history.history["loss"]))
  loss= history.history["loss"]
  val_loss= history.history["accuracy"]

  plt.plot(loss,epochs,label="Train loss")
  plt.plot(val_loss,epochs,label="Validation loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  ## Plot Accuracy
  plt.figure()
  accuracy= history.history["accuracy"]
  val_accuracy= history.history["val_accuracy"]
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();



Writing helper_functions.py


In [5]:
## Download data and unzip it
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip



--2023-08-06 02:12:35--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  78.4MB/s    in 2.1s    

2023-08-06 02:12:37 (78.4 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [6]:
%%writefile -a helper_functions.py

import zipfile
import os
def unzip_data(zipped_file,remove_src_zip=True):
  """
    Takes a zipped file and unzips it.Removes the source zipfile.
  """
  zip_ref= zipfile.ZipFile(zipped_file,"r")
  zip_ref.extractall()
  zip_ref.close()
  os.remove(zipped_file)

Appending to helper_functions.py


In [7]:
from helper_functions import unzip_data
unzip_data("/content/10_food_classes_10_percent.zip")

In [8]:
!rm -r "/content/__MACOSX"

In [9]:
data_dir= "/content/10_food_classes_10_percent"

In [10]:
%%writefile -a helper_functions.py

def walk_through_dir(data_dir):
  for dirpath,dirnames,filenames in os.walk(data_dir):
    print(f" There are {len(filenames)} images and {len(dirnames)} folder at {dirpath}")



Appending to helper_functions.py


In [11]:
from helper_functions import walk_through_dir
walk_through_dir(data_dir)


 There are 0 images and 2 folder at /content/10_food_classes_10_percent
 There are 0 images and 10 folder at /content/10_food_classes_10_percent/test
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/sushi
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/ramen
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/steak
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/fried_rice
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/grilled_salmon
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/pizza
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/chicken_curry
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/hamburger
 There are 250 images and 0 folder at /content/10_food_classes_10_percent/test/ice_cream
 There are 250 images and 0 folder at /content/10_food_

In [12]:
train_dir="/content/10_food_classes_10_percent/train"
test_dir="/content/10_food_classes_10_percent/train"


In [13]:
#Create data inputs
import tensorflow as tf

IMG_SIZE=(224,224)
train_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir
                                                                           ,labels="inferred"
                                                                           ,label_mode="categorical"
                                                                           ,image_size=IMG_SIZE
                                                                           ,shuffle=True
                                                                           )
test_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir
                                                                          ,labels="inferred"
                                                                          ,label_mode="categorical"
                                                                          ,image_size=IMG_SIZE
                                                                          ,shuffle=True)


Found 750 files belonging to 10 classes.
Found 750 files belonging to 10 classes.


In [14]:
train_data_10_percent

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [15]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [38]:
def create_tensorboard_callbacks(log_dir,experiment_name):
  log_dir = log_dir + "/" + experiment_name+ "/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  return tf.keras.callbacks.TensorBoard(log_dir=log_dir)

## Model 0: Build a Transfer Learning model using keras Functional API

Steps:
1. Instantiate a pre-trained base model object by choosing the target model as [EfficientNetB0]() from `tf.keras.applications` setting the `include_top` parameter to `False` ( as we will have our own output layers)
2. Set the base model's `trainable` attribute to `False` to freeze all of the weights in the pre-trained model
3. Define an input layer for our model, what shape of data should our model expects.
4. Normalize the input to our model if needed.
5. Pass the input to the model.
6. Pool the output of the base model into a shape compatible with output activation layer ( turn base model output tensors to same shape as label tensors). This can be done using `tf.keras.layers.GlobalAveragePooling2d()` or `tf.keras.layers.GlobalMaxPooling2D()` though the former is more common in practice.
7. Create an output activation layer using `tf.keras.layers.Dense()` with appropriate activation function and number of neurons.
8. Combine the input and out layers with `tf.keras.Model()`
9. Compile the model using appropriate loss function and optimizer.
10. Fit the model with desired number of epochs and necessary callbacks.

In [39]:
# 1. Instantiate a pre-trained EfficientNetB0 model disabling the top layers with include_top=False
effnetb0_model= tf.keras.applications.EfficientNetB0(include_top=False)
# 2. Set the base model trainable attribute to false to freeze all of the weights in pre-trained model
effnetb0_model.trainable= False
# 3. Define an output layer for our model
inputs= tf.keras.layers.Input(shape=(224,224,3),name="input_layer")
# 4. Normalize the input ( Not required as per this thread: https://github.com/tensorflow/tensorflow/issues/42506)

# 5. Pass the input to the model
x= effnetb0_model(inputs)

# 6. Pool the output of the base model inot a shape compatible with output activation layer
x= tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
# 7. Create the output activation layer
outputs= tf.keras.layers.Dense(len(train_data_10_percent.class_names),activation="softmax",name="output_layer")(x)
# 8. Combine the input and output layers
model= tf.keras.Model(inputs= inputs
                      ,outputs= outputs
                      ,name="effnetb0_pretrained")

# 9. Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy
              ,optimizer=tf.keras.optimizers.Adam()
              ,metrics=["accuracy"])
# 10. Fit the model with desired number of epochs and necessary callbacks.
effnetb0_history= model.fit(train_data_10_percent
                            ,steps_per_epoch=len(train_data_10_percent),
                            validation_data=test_data_10_percent
                            ,validation_steps=int(0.25 * len(test_data_10_percent))
                            ,epochs=5
                            ,callbacks=[create_tensorboard_callbacks(log_dir="transfer_learning",
                                                                     experiment_name="effnet_10_percent_feature_extract")])



Epoch 1/5
24/24 [==============================] - 92s 3s/step - loss: 1.8791 - accuracy: 0.4240 - val_loss: 1.3623 - val_accuracy: 0.7083
Epoch 2/5
24/24 [==============================] - 80s 3s/step - loss: 1.0994 - accuracy: 0.7573 - val_loss: 0.9296 - val_accuracy: 0.7917
Epoch 3/5
24/24 [==============================] - 80s 3s/step - loss: 0.7965 - accuracy: 0.8333 - val_loss: 0.6612 - val_accuracy: 0.8333
Epoch 4/5
24/24 [==============================] - 80s 3s/step - loss: 0.6453 - accuracy: 0.8560 - val_loss: 0.5540 - val_accuracy: 0.8906
Epoch 5/5
24/24 [==============================] - 79s 3s/step - loss: 0.5521 - accuracy: 0.8667 - val_loss: 0.4579 - val_accuracy: 0.8958
